<a href="https://colab.research.google.com/github/muddassirali8089/AASFS/blob/main/sentimentanaltsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# Replace with your actual file path
file_path = '/content/drive/MyDrive/DataSets/Amazon_Unlocked_Mobile.csv'

# Load the dataset
try:
    df = pd.read_csv(file_path)
    print("Data loaded successfully!")
    print(f"Shape of dataframe: {df.shape}")
except Exception as e:
    print(f"Error loading file: {e}")

Data loaded successfully!
Shape of dataframe: (413840, 6)


In [4]:
# Initial checks
print("=== Initial Data Shape ===")
print(df.shape)

print("\n=== Missing Values ===")
print(df.isnull().sum())

print("\n=== Sample Data ===")
display(df.head(3))

=== Initial Data Shape ===
(413840, 6)

=== Missing Values ===
Product Name        0
Brand Name      65171
Price            5933
Rating              0
Reviews            70
Review Votes    12296
dtype: int64

=== Sample Data ===


,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0


In [5]:
# Drop rows with missing reviews (critical for sentiment analysis)
df = df.dropna(subset=['Reviews']).copy()

# Fill missing Brand Names with 'Unknown'
df['Brand Name'] = df['Brand Name'].fillna('Unknown')

# Fill missing Review Votes with 0 (assuming no votes = 0)
df['Review Votes'] = df['Review Votes'].fillna(0).astype(int)

# Remove Price column (not needed)
df = df.drop(columns=['Price'], errors='ignore')

# Verify
print("\n=== After Handling Missing Values ===")
print(f"New Shape: {df.shape}")
print("\nMissing Values:")
print(df.isnull().sum())


=== After Handling Missing Values ===
New Shape: (413770, 5)

Missing Values:
Product Name    0
Brand Name      0
Rating          0
Reviews         0
Review Votes    0
dtype: int64


In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm.auto import tqdm
tqdm.pandas()

# Download required NLTK data with validation
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet', quiet=True)

def clean_text(text):
    # Lowercase
    text = str(text).lower()

    # Remove URLs, special chars, numbers
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize with error handling
    try:
        tokens = nltk.word_tokenize(text)
    except:
        tokens = text.split()  # Fallback to simple whitespace tokenization

    # Lemmatization with WordNet
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if len(word) > 2]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    custom_stopwords = {'phone', 'samsung', 'product', 'item'}
    tokens = [word for word in tokens if word not in stop_words and word not in custom_stopwords]

    return ' '.join(tokens)

# Apply cleaning with progress feedback
print("Cleaning reviews...")
df['Cleaned_Review'] = df['Reviews'].progress_apply(clean_text)

Cleaning reviews...


  0%|          | 0/413770 [00:00<?, ?it/s]

In [8]:
import pandas as pd

# Load the dataset (assuming the file has been uploaded and stored locally)
file_path = "/mnt/data/dataset.csv"
df = pd.read_csv(file_path)

# Drop rows where essential values are missing
df_cleaned = df.dropna(subset=['Product Name', 'Price', 'Rating'])

# Fill missing 'Brand Name' with 'Unknown'
df_cleaned['Brand Name'] = df_cleaned['Brand Name'].fillna('Unknown')

# Fill missing 'Reviews' and 'Review Votes' with 0
df_cleaned['Reviews'] = df_cleaned['Reviews'].fillna(0)
df_cleaned['Review Votes'] = df_cleaned['Review Votes'].fillna(0)

# Convert types
df_cleaned['Reviews'] = df_cleaned['Reviews'].astype(int)
df_cleaned['Review Votes'] = df_cleaned['Review Votes'].astype(int)

# Show the cleaned data
import caas_jupyter_tools as cj
cj.display_dataframe_to_user(name="Cleaned Dataset", dataframe=df_cleaned)

df_cleaned.head()


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/dataset.csv'

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from wordcloud import WordCloud

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline

from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag

import logging
from gensim.models import word2vec
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K
from keras.layers.embeddings import Embedding

/usr/local/lib/python3.11/dist-packages/seaborn/_statistics.py:32: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.2)
  from scipy.stats import gaussian_kde


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
!pip install tqdm

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK data with validation
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet', quiet=True)

def clean_text(text):
    # Lowercase
    text = str(text).lower()

    # Remove URLs, special chars, numbers
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize with error handling
    try:
        tokens = nltk.word_tokenize(text)
    except:
        tokens = text.split()  # Fallback to simple whitespace tokenization

    # Lemmatization with WordNet
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if len(word) > 2]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    custom_stopwords = {'phone', 'samsung', 'product', 'item'}
    tokens = [word for word in tokens if word not in stop_words and word not in custom_stopwords]

    return ' '.join(tokens)

# Apply cleaning with progress feedback
print("Cleaning reviews...")
# Ensure tqdm is imported and pandas integration is enabled before this cell
df['Cleaned_Review'] = df['Reviews'].progress_apply(clean_text)

Cleaning reviews...


  0%|          | 0/413770 [00:00<?, ?it/s]

In [ ]:
# Check cleaned vs original text
print("\n=== Sample Cleaned Reviews ===")
display(df[['Reviews', 'Cleaned_Review']].sample(3))

# Check for empty cleaned reviews
empty_reviews = df[df['Cleaned_Review'].str.strip() == '']
print(f"\nFound {len(empty_reviews)} empty cleaned reviews (to remove)")
df = df[df['Cleaned_Review'].str.strip() != ''].copy()


=== Sample Cleaned Reviews ===


,Reviews,Cleaned_Review
223399,The phone works as expected. Security patch pr...,work expected security patch prevented correct...
59404,It was perfect! It was basically brand new in ...,wa perfect wa basically brand new eye came hea...
53204,Phone came locked that only previous iTunes ac...,came locked previous itunes account holder unl...



Found 3697 empty cleaned reviews (to remove)


In [11]:
print(f"Shape before removing empty reviews: {df.shape}")
df = df[df['Cleaned_Review'].str.strip() != ''].copy()
print(f"Shape after removal: {df.shape}")

Shape before removing empty reviews: (410073, 6)
Shape after removal: (410073, 6)


In [12]:
# Check cleaned vs original text
print("\n=== Sample Cleaned Reviews ===")
display(df[['Reviews', 'Cleaned_Review']].sample(3))

# Check for empty cleaned reviews
empty_reviews = df[df['Cleaned_Review'].str.strip() == '']
print(f"\nFound {len(empty_reviews)} empty cleaned reviews (to remove)")
df = df[df['Cleaned_Review'].str.strip() != ''].copy()


=== Sample Cleaned Reviews ===


,Reviews,Cleaned_Review
124324,very well,well
187123,"This phone is a work of art. It's beautiful, s...",work art beautiful sleek kind thing want perfo...
175506,Good but inside component are exposed such tha...,good inside component exposed likely break com...



Found 0 empty cleaned reviews (to remove)


In [13]:
# Convert Rating to binary sentiment (1=Positive, 0=Negative)
df['Sentiment'] = df['Rating'].apply(lambda x: 1 if x >= 4 else 0)

# Check class balance
print("=== Sentiment Distribution ===")
print(df['Sentiment'].value_counts(normalize=True))

=== Sentiment Distribution ===
Sentiment
1    0.687873
0    0.312127
Name: proportion, dtype: float64


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=5000,          # Limit vocabulary size
    ngram_range=(1, 2),        # Capture phrases like "not good"
    stop_words='english'       # Remove English stopwords
)

X = tfidf.fit_transform(df['Cleaned_Review'])
y = df['Sentiment']

print(f"\nTF-IDF Matrix Shape: {X.shape}")


TF-IDF Matrix Shape: (410073, 5000)


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,            # 20% test set
    random_state=42,          # Reproducibility
    stratify=y                # Preserve class balance
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

Training set: 328058 samples
Test set: 82015 samples


In [21]:
from collections import defaultdict

def extract_keywords(texts, top_n=50):
    word_counts = defaultdict(int)
    for text in texts:
        for word in text.split():
            word_counts[word] += 1
    return sorted(word_counts.items(), key=lambda x: -x[1])[:top_n]

# Get keywords by sentiment
positive_texts = df[df['Sentiment'] == 1]['Cleaned_Review']
negative_texts = df[df['Sentiment'] == 0]['Cleaned_Review']

positive_keywords = extract_keywords(positive_texts)
negative_keywords = extract_keywords(negative_texts)

print("\nTop 5 Positive Keywords:", [word for word, count in positive_keywords[:5]])
print("Top 5 Negative Keywords:", [word for word, count in negative_keywords[:5]])


Top 5 Positive Keywords: ['great', 'wa', 'good', 'work', 'love']
Top 5 Negative Keywords: ['wa', 'work', 'one', 'screen', 'get']


In [22]:
import joblib

artifacts = {
    'tfidf': tfidf,                      # For vectorizing new reviews
    'positive_keywords': positive_keywords,  # For word cloud
    'negative_keywords': negative_keywords,
    'sample_reviews': df.sample(1000)    # For demo
}

joblib.dump(artifacts, '/content/drive/MyDrive/DataSets/sentiment_artifacts.joblib')
df.to_csv('/content/drive/MyDrive/DataSets/cleaned_reviews.csv', index=False)

print("\n=== Artifacts Saved ===")
print("1. TF-IDF Vectorizer")
print("2. Positive/Negative Keywords")
print("3. Sample cleaned reviews (1000 rows)")


=== Artifacts Saved ===
1. TF-IDF Vectorizer
2. Positive/Negative Keywords
3. Sample cleaned reviews (1000 rows)


In [23]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000).fit(X_train, y_train)
print("Test Accuracy:", model.score(X_test, y_test))

Test Accuracy: 0.9061513137840639


In [24]:
from sklearn.linear_model import LogisticRegression

# Train the model
model = LogisticRegression(max_iter=1000).fit(X_train, y_train)

# Save the model
joblib.dump(model, '/content/drive/MyDrive/DataSets/sentiment_model.joblib')
print("Model saved successfully!")

Model saved successfully!


In [25]:
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Load artifacts
artifacts = joblib.load('/content/drive/MyDrive/DataSets/sentiment_artifacts.joblib')
tfidf = artifacts['tfidf']

# Load MODEL (now exists)
model = joblib.load('/content/drive/MyDrive/DataSets/sentiment_model.joblib')

# Test a review
def predict_sentiment(review):
    cleaned = clean_text(review)
    vectorized = tfidf.transform([cleaned])
    pred = model.predict(vectorized)[0]
    proba = model.predict_proba(vectorized)[0]
    print(f"Review: {review}")
    print(f"Sentiment: {'Positive' if pred == 1 else 'Negative'}")
    print(f"Confidence: {max(proba)*100:.2f}%")

predict_sentiment("the display is bad")

Review: the display is bad
Sentiment: Negative
Confidence: 89.61%


In [26]:
# Check cleaned vs original text
print("\n=== Sample Cleaned Reviews ===")
display(df[['Reviews', 'Cleaned_Review']].sample(3))

# Check for empty cleaned reviews
empty_reviews = df[df['Cleaned_Review'].str.strip() == '']
print(f"\nFound {len(empty_reviews)} empty cleaned reviews (to remove)")
df = df[df['Cleaned_Review'].str.strip() != ''].copy()


=== Sample Cleaned Reviews ===


,Reviews,Cleaned_Review
314736,Good s***,good
200981,would give a 5 but there are just a few small ...,would give small thing bug first great doe use...
358641,Very nice phone. Really had no issues with pho...,nice really issue carrier metro pc good compan...



Found 0 empty cleaned reviews (to remove)
